In [1]:
import os
import sys
module_path = "../../src"
if module_path not in sys.path:
    sys.path.append(module_path)

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
from models.ResNetClass import ResNetClass
from utils.label_encoder import label_encoder_target

from torchvision import transforms
from dataset.ImageDataModule import ImageDataModule
from utils.FeatureExtractor import ResNetFeatureExtractor,get_vector_representation

In [3]:
SEED = 42

In [15]:
DATA_PATH = '../../data/higher_modality_vol1.csv'
df = pd.read_csv(DATA_PATH, sep='\t')
le_encoder,dict_label = label_encoder_target(df,target_col='higher_modality')
df.head()

,img,modality,set,source,img_path,higher_modality,caption,split_set
0,11373_2007_9226_Fig1_HTML-10.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-10.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN
1,11373_2007_9226_Fig1_HTML-11.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-11.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN
2,11373_2007_9226_Fig1_HTML-12.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-12.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN
3,11373_2007_9226_Fig1_HTML-13.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-13.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN
4,11373_2007_9226_Fig1_HTML-14.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-14.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN


In [46]:
from os import path
import pandas as pd

PROJECTIONS = '/mnt/artifacts/projections'
CLASSIFIERS = '/mnt/artifacts/classifiers'

In [44]:
def fetch_projection(classifier, version, projection):
    file_name = f"{classifier}_{version}.csv"
    file_path = path.join(PROJECTIONS, file_name)
    print(file_path)

In [45]:
fetch_projection("test", "v1", "umap")

/mnt/artifacts/projections/test_v1.csv


In [53]:
def file2json(file_path, projection):
    df = pd.read_csv(file_path)
    x = f"{projection}_0"
    y = f"{projection}_1"
    df = df[df["img", x, y]]
    return df.to_json(orient="records")

In [87]:
df = pd.read_csv('/mnt/artifacts/projections/higher_modality_v1.csv', sep='\t')
df.head()

,img,modality,set,source,img_path,higher_modality,caption,split_set,feature_vector,pca_x,pca_y,umap_x,umap_y
0,11373_2007_9226_Fig1_HTML-10.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-10.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,[0.03698349 0.02797039 0.06378924 ... 0.605288...,2.800454,15.985010,7.017618,4.759446
1,11373_2007_9226_Fig1_HTML-11.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-11.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,[0.11966821 0.03156081 0.14387487 ... 0.531013...,2.920290,17.780977,7.123211,4.764099
2,11373_2007_9226_Fig1_HTML-12.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-12.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,[0.02774515 0.19340134 0.14208017 ... 0.124674...,2.667454,17.838985,7.782630,6.780328
3,11373_2007_9226_Fig1_HTML-13.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-13.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,[0.08631085 0.25344065 0.58346796 ... 0.763252...,-3.080419,14.389732,7.719876,4.297488
4,11373_2007_9226_Fig1_HTML-14.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-14.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,[0.28187314 0.04344651 0.8873729 ... 0.408760...,0.902185,15.456240,6.919820,4.794608


In [83]:
X = df.feature_vector.values

In [88]:
import ast
X = df.feature_vector.apply(ast.literal_eval)

SyntaxError: invalid syntax (<unknown>, line 1)

In [96]:
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

KNeighborsClassifier(n_neighbors=3)

In [99]:
A = neigh.kneighbors_graph([[0]])
A.toarray()

array([[1., 1., 1., 0.]])

In [112]:
from sklearn.neighbors import NearestNeighbors
samples = [[0, 1], [0, 5], [10, 0]]

neigh = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(samples)

In [115]:
neigh.kneighbors(samples, 3, return_distance=False)

array([[0, 1, 2],
       [1, 0, 2],
       [2, 0, 1]])

In [123]:
df.iloc[[100, 1, 10]].higher_modality.values

array(['GRAPHICS', 'MICROSCOPY', 'MICROSCOPY'], dtype=object)

In [47]:
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

def calc_neighborhood_hit(df, projection, label_col, n_neighbors=6):
    x = f"{projection}_x"
    y = f"{projection}_y"
    
    projections = [[i, j] for (i, j) in zip(df[x].values, df[y].values)]
    neigh = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(projections)
    le = preprocessing.LabelEncoder().fit(df[label_col].unique())
    
    n_hits = []
    for neighborhood in neigh.kneighbors(projections, n_neighbors + 1, return_distance=False):
        labels  = le.transform(df.iloc[neighborhood][label_col].values)
        targets = [labels[0]] * (len(labels) - 1) 
        n_hit = np.mean(targets == labels[1:])
        n_hits.append(n_hit)
    return n_hits

In [17]:
import pandas as pd
df_parquet = pd.read_parquet('/mnt/artifacts/projections/higher_modality_v1.parquet')
df_parquet.head()

,img,modality,set,source,img_path,higher_modality,caption,split_set,feature_vector,pca_x,pca_y,umap_x,umap_y
0,11373_2007_9226_Fig1_HTML-10.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-10.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,"[0.03698349, 0.02797039, 0.06378924, 0.0206936...",2.800454,15.985010,7.017618,4.759446
1,11373_2007_9226_Fig1_HTML-11.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-11.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,"[0.11966821, 0.03156081, 0.14387487, 0.0154373...",2.920290,17.780977,7.123211,4.764099
2,11373_2007_9226_Fig1_HTML-12.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-12.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,"[0.027745148, 0.19340134, 0.14208017, 0.083299...",2.667454,17.838985,7.782630,6.780328
3,11373_2007_9226_Fig1_HTML-13.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-13.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,"[0.08631085, 0.25344065, 0.58346796, 0.1590379...",-3.080419,14.389732,7.719876,4.297488
4,11373_2007_9226_Fig1_HTML-14.jpg,DMFL,TRAIN,clef16,2016/train/DMFL/11373_2007_9226_Fig1_HTML-14.jpg,MICROSCOPY,"Colocalization of hNopp140, pol I and rDNA rep...",TRAIN,"[0.28187314, 0.04344651, 0.8873729, 0.21146952...",0.902185,15.456240,6.919820,4.794608


In [18]:
A = df_parquet[["img", "img_path", "higher_modality", "umap_x", "umap_y"]].rename(columns={"higher_modality": "label"})

In [55]:
df_parquet.split_set.unique()

array(['TRAIN', 'VAL', 'TEST'], dtype=object)

In [19]:
A.head()

,img,img_path,label,umap_x,umap_y
0,11373_2007_9226_Fig1_HTML-10.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-10.jpg,MICROSCOPY,7.017618,4.759446
1,11373_2007_9226_Fig1_HTML-11.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-11.jpg,MICROSCOPY,7.123211,4.764099
2,11373_2007_9226_Fig1_HTML-12.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-12.jpg,MICROSCOPY,7.782630,6.780328
3,11373_2007_9226_Fig1_HTML-13.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-13.jpg,MICROSCOPY,7.719876,4.297488
4,11373_2007_9226_Fig1_HTML-14.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-14.jpg,MICROSCOPY,6.919820,4.794608


In [48]:
hits = calc_neighborhood_hit(A, "umap", "label")

In [51]:
A['hits'] = hits

In [52]:
A.head()

,img,img_path,label,umap_x,umap_y,hits
0,11373_2007_9226_Fig1_HTML-10.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-10.jpg,MICROSCOPY,7.017618,4.759446,1.0
1,11373_2007_9226_Fig1_HTML-11.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-11.jpg,MICROSCOPY,7.123211,4.764099,1.0
2,11373_2007_9226_Fig1_HTML-12.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-12.jpg,MICROSCOPY,7.782630,6.780328,1.0
3,11373_2007_9226_Fig1_HTML-13.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-13.jpg,MICROSCOPY,7.719876,4.297488,1.0
4,11373_2007_9226_Fig1_HTML-14.jpg,2016/train/DMFL/11373_2007_9226_Fig1_HTML-14.jpg,MICROSCOPY,6.919820,4.794608,1.0
